# Gründe für Ausfälle der Zählstellen

In [1]:
import pandas as pd
import altair as alt

In [2]:
# Aktuelle Pfade
file_paths = [
     "data/rad_2022_tage_19_06_23_r.csv",
     "data/rad_2021_tage_19_06_23_r.csv",
     "data/rad_2020_tage_19_06_23_r.csv",
     "data/rad_2019_tage_19_06_23_r.csv",
     "data/rad_2018_tage_19_06_23_r.csv",
     "data/rad_2017_tage_19_06_23_r.csv",
     "data/rad_2016_tage_19_06_23_r.csv",
     "data/rad_2015_tage_19_06_23_r.csv",
     "data/rad_2014_tage_19_06_23_r.csv",
     "data/rad_2013_tage_19_06_23_r.csv",
     "data/rad_2012_tage_19_06_23_r.csv",
     "data/rad_2011_tage_19_06_23_r.csv",
     "data/rad_2010_tage_19_06_23_r.csv",
     "data/rad_2009_tage_19_06_23_r.csv",
     "data/rad_2008_tage_19_06_23_r.csv"
]

In [3]:
df_list = []

for file_path in file_paths:
    df_temp = pd.read_csv(file_path)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True)

In [4]:
df.head()

,datum,zaehlstelle,uhrzeit_start,uhrzeit_ende,richtung_1,richtung_2,gesamt,min.temp,max.temp,niederschlag,bewoelkung,sonnenstunden,kommentar
0,2022-01-01,Arnulf,00:00,23:59,444.0,0.0,444.0,3.2,13.4,0.0,68,7.6,NaN
1,2022-01-02,Arnulf,00:00,23:59,476.0,0.0,476.0,1.9,15.8,0.0,80,4.7,NaN
2,2022-01-03,Arnulf,00:00,23:59,752.0,0.0,752.0,8.7,11.5,0.7,99,0.0,NaN
3,2022-01-04,Arnulf,00:00,23:59,603.0,0.0,603.0,3.7,16.5,10.3,98,1.0,NaN
4,2022-01-05,Arnulf,00:00,23:59,578.0,0.0,578.0,0.3,3.9,1.2,95,0.4,NaN


# Vorverarbeitung der Daten: Ausfallgründe aus Kommentarspalte ermittelt

In [5]:
# Spalte 'gesamt' auf NA-Werte überprüfen
na_in_gesamt = df['gesamt'].isna() # True, wenn NA in gesamt vorhanden

# Spalte 'gesamt' auf Nullwerte überprüfen
null_in_gesamt = df['gesamt'] == 0 # True, wenn value=0 in gesamt

# Erstellen Sie einen DataFrame aus 'kommentar' und 'datum' Werten, wenn 'gesamt' NA oder Null ist
kommentar_und_datum_fuer_na_oder_null_gesamt = df.loc[na_in_gesamt | null_in_gesamt, ['kommentar', 'datum', 'zaehlstelle']]

# Anzahl der einzigartigen Werte in der Spalte 'kommentar' ermitteln
anzahl_einzigartige_werte_kommentar = kommentar_und_datum_fuer_na_oder_null_gesamt['kommentar'].nunique()
print("Anzahl der einzigartigen Werte in 'kommentar': ", anzahl_einzigartige_werte_kommentar)

# Zählt die einzigartigen Werte in der Spalte 'kommentar'
zaehlung_kommentare = kommentar_und_datum_fuer_na_oder_null_gesamt['kommentar'].value_counts()
print(zaehlung_kommentare)

Anzahl der einzigartigen Werte in 'kommentar':  6
Zählstelle noch nicht in Betrieb                                          1028
Baustelle                                                                  609
Ausfall nach Beschädigung                                                   82
Radweg vereist / nach Schneefall nicht geräumt / keine Messung möglich      48
Ausfall                                                                     38
Austausch Sensor                                                             2
Name: kommentar, dtype: int64


# Basisdiagramm - ohne Sortierung, ohne Labels

In [6]:
source = pd.DataFrame({
  'kommentar': zaehlung_kommentare.index,
  'zaehlung': zaehlung_kommentare.values,
})

pie = alt.Chart(source).mark_arc().encode(
    alt.Theta('zaehlung:Q', stack=True),
    alt.Color('kommentar:N', legend=alt.Legend(title='Werte der Spalte "kommentar"',  orient="right")),
    alt.Tooltip('zaehlung:Q')
)

pie

alt.Chart(...)

# Diagramm - sortiert nach Größe, ohne Labels

In [9]:
source = pd.DataFrame({
    'kommentar': zaehlung_kommentare.index,
    'zaehlung': zaehlung_kommentare.values,
})

# Berechne prozentuale Anteile
prozentanteil = ((zaehlung_kommentare / zaehlung_kommentare.sum()) * 100).round(2)
#print(prozentanteil)

sortierte_anteile = sorted(prozentanteil)
#print(sortierte_anteile)

# Erstelle DataFrame für das Chart
chart_data = pd.DataFrame({
    'Kommentar': prozentanteil.index,
    'Prozentanteil': sortierte_anteile
})

# Erstelle Pie Chart mit aufsteigender Sortierung nach Anteil
chart = alt.Chart(chart_data).mark_arc().encode(
    alt.Theta('Prozentanteil:Q', sort=alt.SortOrder('descending')),
    color='Kommentar:N',
    tooltip=['Kommentar', 'Prozentanteil']
).properties(
    width=500,
    height=400,
    title='Verteilung der Kommentare nach Prozentanteil'
)

text = alt.Chart(chart_data).mark_text(
    align='center',
    baseline='middle',
    radius=200,  # Adjust the radius to move labels outside the pie chart
    color='black',  # Set label color to black
    fontSize=14  # Adjust the font size as needed
)

# Combine chart and labels
pie = chart + text
pie

alt.LayerChart(...)

# Betrachtung der Werte

In [8]:
# Spalte 'gesamt' auf NA-Werte überprüfen
na_in_gesamt = df['gesamt'].isna() # True, wenn NA in gesamt vorhanden

# Spalte 'gesamt' auf Nullwerte überprüfen
null_in_gesamt = df['gesamt'] == 0 # True, wenn value=0 in gesamt

# Erstellen Sie einen DataFrame aus 'kommentar' und 'datum' Werten, wenn 'gesamt' NA oder Null ist
kommentar_und_datum_fuer_na_oder_null_gesamt = df.loc[na_in_gesamt | null_in_gesamt, ['kommentar', 'datum', 'zaehlstelle']]

# Anzahl der einzigartigen Werte in der Spalte 'kommentar' ermitteln
anzahl_einzigartige_werte_kommentar = kommentar_und_datum_fuer_na_oder_null_gesamt['kommentar'].nunique()
print("Anzahl der einzigartigen Werte in 'kommentar': ", anzahl_einzigartige_werte_kommentar)

# Zählt die einzigartigen Werte in der Spalte 'kommentar'
zaehlung_kommentare = kommentar_und_datum_fuer_na_oder_null_gesamt['kommentar'].value_counts()
print(zaehlung_kommentare)

zaehlung_kommentare.index
zaehlung_kommentare.values

Anzahl der einzigartigen Werte in 'kommentar':  6
Zählstelle noch nicht in Betrieb                                          1028
Baustelle                                                                  609
Ausfall nach Beschädigung                                                   82
Radweg vereist / nach Schneefall nicht geräumt / keine Messung möglich      48
Ausfall                                                                     38
Austausch Sensor                                                             2
Name: kommentar, dtype: int64


array([1028,  609,   82,   48,   38,    2])

# Vollständiges Diagramm

In [10]:
# Berechne prozentuale Anteile
prozentanteil = ((zaehlung_kommentare / zaehlung_kommentare.sum()) * 100).round(2)
#print(prozentanteil)

sortierte_anteile = sorted(prozentanteil)
#print(sortierte_anteile)

# Erstelle DataFrame für das Chart
chart_data = pd.DataFrame({
    'Kommentar': prozentanteil.index,
    'Prozentanteil': sortierte_anteile
})

chart_data['radius_condition'] = chart_data['Prozentanteil'] < 1

base = alt.Chart(chart_data).encode(
    theta=alt.Theta("Prozentanteil:Q").stack(True),
    color=alt.Color("Kommentar:N", title='Ausfallgründe'),
).properties(
    title='Verteilung der Ausfallgründe in Prozent'
)

# Erhöhe den Radius im mark_text Layer nur für Prozentsätze < 1
pie = base.mark_arc(outerRadius=115)
text = base.mark_text(size=12).encode(
    text="Prozentanteil:Q",
    radius=alt.Radius(
        'radius_condition:N',
         scale=alt.Scale(domain=[True, False], range=[130, 150])
    )
)

# Kombiniere das Pie-Diagramm, Textlabels und die Legende
final_chart = (pie + text)

# Zeige das Diagramm
final_chart


alt.LayerChart(...)

In [20]:
# Berechne prozentuale Anteile
prozentanteil = ((zaehlung_kommentare / zaehlung_kommentare.sum()) * 100).round(2)

# Konvertiere die Werte in numerische Werte vor der Sortierung
prozentanteil = pd.to_numeric(prozentanteil, errors='coerce')

# Sortiere den DataFrame nach dem Prozentanteil in absteigender Reihenfolge
sortierte_anteile = prozentanteil.sort_values(ascending=True)

# Erstelle DataFrame für das Chart
chart_data = pd.DataFrame({
    'Kommentar': sortierte_anteile.index,
    'Prozentanteil': sortierte_anteile.values
})

chart_data['radius_condition'] = chart_data['Prozentanteil'] < 1

# Sortiere die Kategorien nach dem sortierten Index
base = alt.Chart(chart_data).encode(
    theta=alt.Theta("Prozentanteil:Q").stack(True),
    color=alt.Color("Kommentar:N", title='Ausfallgründe', sort=alt.EncodingSortField(field="sort_index", order='ascending')),
).properties(
    title='Verteilung der Ausfallgründe in Prozent'
)


# Erhöhe den Radius im mark_text Layer nur für Prozentsätze < 1
pie = base.mark_arc(outerRadius=115)
text = base.mark_text(size=12).encode(
    text="Prozentanteil:Q",
    radius=alt.Radius(
        'radius_condition:N',
         scale=alt.Scale(domain=[True, False], range=[130, 150])
    )
)

# Kombiniere das Pie-Diagramm, Textlabels und die Legende
final_chart = (pie + text)

# Zeige das Diagramm
final_chart


alt.LayerChart(...)